In [1]:
import pandas as pd
import rltk
import re
from difflib import SequenceMatcher
from tqdm import tqdm
import json
import os

In [2]:
os.listdir('../data')

['semantic_joined_dblp.json.zip',
 '.DS_Store',
 'Merged_graphics.csv',
 'semantic_joined_dblp.json',
 'Merged_ai.csv',
 'output.json',
 'semantic_scholar_articles_20000.csv',
 'Merged_vision.csv',
 'Merged_nlp.csv',
 'output_compiled.json']

## Loading Semantic and DBLP JSON

In [3]:
f = open('../data/semantic_joined_dblp.json','r')
t=f.read()
f.close()
js = json.loads(t)
df_js = pd.DataFrame(js)

In [4]:
df_js.head()

,abstract,arxivId,authors,citationVelocity,citations,corpusId,doi,influentialCitationCount,paperId,references,title,topics,url,year,dblp_info
0,The artificial neural networks that are used t...,NaN,"[{'authorId': '1695689', 'name': 'Geoffrey E. ...",150,"[{'arxivId': None, 'authors': [{'authorId': '1...",6138085,10.1007/978-3-642-21735-7_6,52,20f0357688876fa4662f806f985779dce6e24f3c,"[{'arxivId': None, 'authors': [{'authorId': '5...",Transforming Auto-Encoders,"[{'topic': 'Computer vision', 'topicId': '5332...",https://www.semanticscholar.org/paper/20f03576...,2011.0,"{'@score': '3', '@id': '3107620', 'info': {'au..."
1,In order to improve the accuracy of capsule ne...,NaN,"[{'authorId': '144866658', 'name': 'Xin Ning',...",0,[],215723800,10.1109/ACCESS.2020.2982782,0,4c79e754407d41904979746fbc8090545c0aeec9,"[{'arxivId': '1810.10183', 'authors': [{'autho...",BDARS_CapsNet: Bi-Directional Attention Routin...,"[{'topic': 'Routing', 'topicId': '1048', 'url'...",https://www.semanticscholar.org/paper/4c79e754...,2020.0,NaN
2,The current dominant paradigm for feature lear...,1505.01596,"[{'authorId': '33932184', 'name': 'Pulkit Agra...",96,"[{'arxivId': None, 'authors': [{'authorId': '2...",1637703,10.1109/ICCV.2015.13,31,dfbfaaec46d38392f61d683c340ee92a0a66e5d9,"[{'arxivId': None, 'authors': [{'authorId': '2...",Learning to See by Moving,"[{'topic': 'Visual odometry', 'topicId': '644'...",https://www.semanticscholar.org/paper/dfbfaaec...,2015.0,"{'@score': '5', '@id': '2026007', 'info': {'au..."
3,"In this work, we address the problem of improv...",NaN,"[{'authorId': '3458345', 'name': 'Zhun Sun', '...",0,"[{'arxivId': None, 'authors': [{'authorId': '3...",53872389,10.1109/CVPR.2018.00830,2,51bd5966fc992498cb1147d34527e33656c696bb,"[{'arxivId': None, 'authors': [{'authorId': '2...",Feature Quantization for Defending Against Dis...,"[{'topic': 'Distortion', 'topicId': '15080', '...",https://www.semanticscholar.org/paper/51bd5966...,2018.0,"{'@score': '8', '@id': '1024764', 'info': {'au..."
4,"We introduce BSD-GAN, a novel multi-branch and...",1803.08467,"[{'authorId': '39737792', 'name': 'Zili Yi', '...",0,"[{'arxivId': '2009.13311', 'authors': [{'autho...",220666238,10.1109/TIP.2020.3014608,0,70977b9464b94f71c2c4fc68aa03082e5739b6c3,"[{'arxivId': '1710.10196', 'authors': [{'autho...",BSD-GAN: Branched Generative Adversarial Netwo...,"[{'topic': 'BSD', 'topicId': '10760', 'url': '...",https://www.semanticscholar.org/paper/70977b94...,2020.0,"{'@score': '15', '@id': '211133', 'info': {'au..."


In [5]:
ls = [i for i in os.listdir('../data') if "Merged" in i]

In [6]:
df = pd.DataFrame()
for i in ls:
    df_2 = pd.read_csv('../data/'+i)
    df = df.append(df_2)

In [7]:
df.head()

,ID,title,authors,published,updated,abstract,categories,citations,arxiv_url,gscholar_url,journal
0,0,Dynamic Graph CNN for Learning on Point Clouds,"['Yue Wang', 'Sanjay E. Sarma', 'Justin M. Sol...",2018-01-24T01:14:04Z,2019-06-11T06:11:21Z,Point clouds provide a flexible geometric repr...,['cs.CV'],536.0,http://arxiv.org/abs/1801.07829v2,http://scholar.google.com/scholar?oi=bibs&clus...,"ACM Transactions on Graphics (TOG) 38 (5), 1-12"
1,1,Looking to Listen at the Cocktail Party: A Spe...,"['Avinatan Hassidim', 'Oran Lang', 'Michael Ru...",2018-04-10T16:28:59Z,2018-08-09T21:22:37Z,We present a joint audio-visual model for isol...,"['cs.SD', 'cs.CV', 'eess.AS']",189.0,http://arxiv.org/abs/1804.03619v2,http://scholar.google.com/scholar?oi=bibs&clus...,"ACM Transactions on Graphics (TOG) 37 (4), 1-11"
2,2,Convergence of univariate non-stationary subdi...,"['Carla Manni', 'Costanza Conti', 'Marie-Laure...",2014-10-10T10:30:14Z,2014-10-10T10:30:14Z,A new equivalence notion between non-stationar...,['math.NA'],34.0,http://arxiv.org/abs/1410.2729v1,http://scholar.google.com/scholar?oi=bibs&clus...,"Computer Aided Geometric Design 37, 1-8"
3,3,HDR image reconstruction from a single exposur...,"['Joel Kronander', 'Rafał K. Mantiuk', 'Gyorgy...",2017-10-20T10:48:22Z,2017-10-20T10:48:22Z,Camera sensors can only capture a limited rang...,"['cs.CV', 'cs.GR', 'cs.LG']",145.0,http://arxiv.org/abs/1710.07480v1,http://scholar.google.com/scholar?oi=bibs&clus...,"ACM Transactions on Graphics (TOG) 36 (6), 1-15"
4,4,Learning Style Similarity for Searching Infogr...,"['Aaron Hertzmann', 'Mira Dontcheva', 'Babak S...",2015-05-05T22:59:32Z,2015-05-05T22:59:32Z,Infographics are complex graphic designs integ...,"['cs.GR', 'cs.CV', 'cs.HC', 'cs.IR', 'cs.MM']",24.0,http://arxiv.org/abs/1505.01214v1,http://scholar.google.com/scholar?oi=bibs&clus...,Proceedings of the 41st Graphics Interface Con...


## Load Data

### Naïve Data Merge

In [8]:
# How many matches can be found with a naÏve identical string approach?
d1 = df_js['title'].values
d2 = df['title'].values
print(len([1 for w in d1 if w in d2]))

159


## RLTK Data Merge

In [9]:
# RLTK Tokenizer
tokenizer = rltk.CrfTokenizer()

**Arxiv/Google Scholar Dataset**

In [10]:
df_js.reset_index(inplace = True)

In [11]:
df_js.keys()

Index(['index', 'abstract', 'arxivId', 'authors', 'citationVelocity',
       'citations', 'corpusId', 'doi', 'influentialCitationCount', 'paperId',
       'references', 'title', 'topics', 'url', 'year', 'dblp_info'],
      dtype='object')

In [12]:
df.reset_index(inplace=True)

In [13]:
df.keys()

Index(['index', 'ID', 'title', 'authors', 'published', 'updated', 'abstract',
       'categories', 'citations', 'arxiv_url', 'gscholar_url', 'journal'],
      dtype='object')

In [14]:
import datetime

In [15]:
df_js['authors'][0]

"[{'authorId': '1695689', 'name': 'Geoffrey E. Hinton', 'url': 'https://www.semanticscholar.org/author/1695689'}, {'authorId': '2064160', 'name': 'A. Krizhevsky', 'url': 'https://www.semanticscholar.org/author/2064160'}, {'authorId': '49185042', 'name': 'S. Wang', 'url': 'https://www.semanticscholar.org/author/49185042'}]"

In [16]:
df['authors'][0]

"['Yue Wang', 'Sanjay E. Sarma', 'Justin M. Solomon', 'Yongbin Sun', 'Ziwei Liu', 'Michael M. Bronstein']"

In [17]:
import ast

In [18]:
def getPlainString(string):
    return ''.join([s for s in string if not s.isnumeric()]).strip()


In [19]:
class ArxivRecord(rltk.Record):
    def __init__(self, raw_object):
        super().__init__(raw_object)
        self.name = 'ArxivRecord'
        
    @property
    def id(self):
        return str(self.raw_object['ID'])
    
    @rltk.cached_property
    def authors_string(self):
        return self.raw_object['authors']
    
    @rltk.cached_property
    def title_string(self):
        return self.raw_object['title']
        
#     @rltk.cached_property
#     def summary_string(self):
#         return self.raw_object['summary']
    
    @rltk.cached_property
    def categories_string(self):
        return self.raw_object['categories']
    
    @rltk.cached_property
    def published_string(self):
        return self.raw_object['published']
    
    @rltk.cached_property
    def updated_string(self):
        return self.raw_object['updated']
        
    @rltk.cached_property
    def blocking_author_tokens(self):
        return set([getPlainString(s).split(' ')[-1] for s in ast.literal_eval(self.raw_object['authors'])])
    
#     @rltk.cached_property
#     def url_string(self):
#         return self.raw_object['url']
    
    @rltk.cached_property
    def blocking_tokens(self):
        tokens = ' '.join([self.title_string])
        tokens = re.sub(r'\bThe\b', '', tokens)
        tokens = re.sub(r'\bthe\b', '', tokens)
        tokens = re.sub(r'\bof\b', '', tokens)
        tokens = re.sub(r"\b's\b", '', tokens)
        tokens = re.sub(r'\band\b', '', tokens)
        tokens = re.sub(r'\bI\b', '', tokens)
        tokens = re.sub(r'\bA\b', '', tokens)
        tokens = re.sub(r'\bin\b', '', tokens)
        tokens = re.sub(r'\bfor\b', '', tokens)
        tokens = re.sub(r'\bon\b', '', tokens)
        tokens = re.sub(r'\bwith\b', '', tokens)
        return set(tokenizer.tokenize(tokens))

In [20]:
ds_1 = rltk.Dataset(reader=rltk.DataFrameReader(df), record_class=ArxivRecord, adapter=rltk.MemoryKeyValueAdapter())
print(type(ds_1))
ds_1.generate_dataframe().head(3)

<class 'rltk.dataset.Dataset'>


,id,authors_string,title_string,categories_string,published_string,updated_string,blocking_author_tokens,blocking_tokens
0,0,"['Barret Zoph', 'Kevin Knight']",Multi-Source Neural Translation,['cs.CL'],2016-01-05T00:49:22Z,2016-01-05T00:49:22Z,"{Zoph, Knight}","{-, Neural, Multi, Source, Translation}"
1,1,"['Xiaodong He', 'Asli Celikyilmaz', 'Antoine B...",Deep Communicating Agents for Abstractive Summ...,['cs.CL'],2018-03-27T23:29:23Z,2018-08-15T18:54:22Z,"{Celikyilmaz, He, Choi, Bosselut}","{Summarization, Abstractive, Agents, Communica..."
2,2,"['Furu Wei', 'Ming Zhou', 'Nan Yang', 'Qingyu ...",Selective Encoding for Abstractive Sentence Su...,['cs.CL'],2017-04-24T07:57:37Z,2017-04-24T07:57:37Z,"{Yang, Wei, Zhou}","{Encoding, Summarization, Abstractive, Sentenc..."


**Google Scholar Dataset**

In [21]:
class SemanticRecord(rltk.Record):
    def __init__(self, raw_object):
        super().__init__(raw_object)
        self.name = 'SemanticRecord'
        
    @property
    def id(self):
        return str(self.raw_object['index'])
    
    @rltk.cached_property
    def authors_string(self):
        return self.raw_object['authors']
    
    @rltk.cached_property
    def title_string(self):
        return self.raw_object['title']
        
#     @rltk.cached_property
#     def journal_string(self):
#         return self.raw_object['journal']
    
    @rltk.cached_property
    def citations_string(self):
        return self.raw_object['citations']
    
    @rltk.cached_property
    def year_string(self):
        return self.raw_object['year']
    
    @rltk.cached_property
    def url_string(self):
        return self.raw_object['url']
    
    
    @rltk.cached_property
    def blocking_author_tokens(self):
        return set([getPlainString(s['name']).split(' ')[-1] for s in ast.literal_eval((self.raw_object['authors']))])

    
    @rltk.cached_property
    def blocking_tokens(self):
        tokens = ' '.join([self.title_string])
        tokens = re.sub(r'\bThe\b', '', tokens)
        tokens = re.sub(r'\bthe\b', '', tokens)
        tokens = re.sub(r'\bof\b', '', tokens)
        tokens = re.sub(r"\b's\b", '', tokens)
        tokens = re.sub(r'\band\b', '', tokens)
        tokens = re.sub(r'\bI\b', '', tokens)
        tokens = re.sub(r'\bA\b', '', tokens)
        tokens = re.sub(r'\bin\b', '', tokens)
        tokens = re.sub(r'\bfor\b', '', tokens)
        tokens = re.sub(r'\bon\b', '', tokens)
        tokens = re.sub(r'\bwith\b', '', tokens)
        return set(tokenizer.tokenize(tokens))

In [22]:
ds_2 = rltk.Dataset(reader=rltk.DataFrameReader(df_js), record_class=SemanticRecord, adapter=rltk.MemoryKeyValueAdapter())
print(type(ds_2))
ds_2.generate_dataframe().head(3)

<class 'rltk.dataset.Dataset'>


,id,authors_string,title_string,citations_string,year_string,url_string,blocking_author_tokens,blocking_tokens
0,0,"[{'authorId': '1695689', 'name': 'Geoffrey E. ...",Transforming Auto-Encoders,"[{'arxivId': None, 'authors': [{'authorId': '1...",2011.0,https://www.semanticscholar.org/paper/20f03576...,"{Wang, Hinton, Krizhevsky}","{Auto, -, Transforming, Encoders}"
1,1,"[{'authorId': '144866658', 'name': 'Xin Ning',...",BDARS_CapsNet: Bi-Directional Attention Routin...,[],2020.0,https://www.semanticscholar.org/paper/4c79e754...,"{Li, Tian, Ning, Sun, Nie, Chen, Lu}","{Network, Directional, Capsule, _, CapsNet, -,..."
2,2,"[{'authorId': '33932184', 'name': 'Pulkit Agra...",Learning to See by Moving,"[{'arxivId': None, 'authors': [{'authorId': '2...",2015.0,https://www.semanticscholar.org/paper/dfbfaaec...,"{Agrawal, Malik, Carreira}","{to, See, by, Moving, Learning}"


### Blocking

In [23]:
# Generate blocks from tokens
token_blocker = rltk.TokenBlockGenerator()
blocks = token_blocker.generate(
    token_blocker.block(ds_1, property_='blocking_author_tokens'),
    token_blocker.block(ds_2, property_='blocking_author_tokens'))
print(type(blocks))

<class 'rltk.blocking.block.Block'>


In [24]:
# Extract all record pairs from the block
record_pairs = rltk.get_record_pairs(ds_1, ds_2, block=blocks)

# Get the total number of record pairs generated
compared_pairs = len(list(record_pairs))

# Get the number of elements in each rltk.Dataset
tally_imdb = ds_1.generate_dataframe().shape[0]
tally_tmd = ds_2.generate_dataframe().shape[0]

# Calculate the total number of pairs if both datasets were to be compared without any blocking (eg: a double for loop)
tally_unblocked = tally_imdb * tally_tmd

# Calculate how much smaller the blocked pairings are
reduction_ratio = compared_pairs / tally_unblocked

3036085/(1-0.93040)

# Calculate the reduction ratio (the inverse of the )
reduction_ratio = 1 - reduction_ratio
print(f'Reduction Ratio: {reduction_ratio:.5f}')

Reduction Ratio: 0.93040


In [ ]:
Total Pairs = 43621910.9
Compared Pairs = 3036085
Reduction Ratio = 0.93040

In [25]:
compared_pairs

3036085

### Matching Functions

In [26]:
def title_similarity(arxiv_tuple, gscholar_tuple):
    arxiv_title = arxiv_tuple.title_string.strip().lower()
    gscholar_title = gscholar_tuple.title_string.strip().lower()
    similarity = SequenceMatcher(None, arxiv_title, gscholar_title).ratio()

    penalties = sum([len(arxiv_title)<=6,
                     len(gscholar_title)<=6])

    return similarity * (0.9**penalties)

In [27]:
def author_similarity(arxiv_tuple, gscholar_tuple):
    arxiv_author = ' '.join(arxiv_tuple.authors_string).strip().lower()
    gscholar_author = ' '.join(gscholar_tuple.authors_string).strip().lower()
    similarity = SequenceMatcher(None, arxiv_author, gscholar_author).ratio() 
    return similarity

In [28]:
def year_similarity(arxiv_tuple, gscholar_tuple):
#     arxiv_year = int(float(arxiv_tuple.updated_string[0:4]))
#     print(arxiv_tuple.published_string)
    if str(arxiv_tuple.published_string) == "nan":
        return 0
    arxiv_year = datetime.datetime.strptime(arxiv_tuple.published_string, '%Y-%m-%dT%H:%M:%SZ').year

    gscholar_year = int(float(gscholar_tuple.year_string))
    similarity = 1 /(1 + abs(arxiv_year-gscholar_year))
    return similarity

In [29]:
def elementwise_similarity(arxiv_tuple, gscholar_tuple, match_threshold=0.75):
    sim_title = title_similarity(arxiv_tuple, gscholar_tuple)
    sim_author = author_similarity(arxiv_tuple, gscholar_tuple)
    sim_year = year_similarity(arxiv_tuple, gscholar_tuple)

    element_similarity = (0.70 * sim_title) + (0.15 * sim_author) + (0.15 * sim_year)

    return element_similarity > match_threshold, element_similarity

In [36]:
# Predict matches for all pairs in the blocked data 
print(f'Arxiv samples: {df.shape[0]}')
print(f'Semantic samples: {df_js.shape[0]}')

summary_df = pd.DataFrame()
THRESHOLDS = [T/100 for T in range(65, 101, 5)]

# Iterate through various thresholds to find the most matches without any duplicates
for T in tqdm(THRESHOLDS):
#     c=0

    # Set to store pairs of IDs matched
    ids_matched = set()
    
    # Iterate through candidates on the block
    for block_id, arxiv_id, gscholar_id in blocks.pairwise(ds_1, ds_2):
        
        # Find similarity at a given threshold
        match , similarity = elementwise_similarity(ds_1.get_record(arxiv_id),
                                                    ds_2.get_record(gscholar_id),
                                                    match_threshold=T)
        # If a match is found, add to the set of matches
        if match:
            ids_matched.add((arxiv_id, gscholar_id))
            
#         c=c+1
#         if c>1000:
#             break
    # Count the number of unique elements derived from each source
    set_a = set()
    set_b = set()
    for tp in ids_matched:
        set_a.add(tp[0])
        set_b.add(tp[1])
    
    summary_df.at[T, 'Matches'] = int(len(ids_matched))
    summary_df.at[T, 'Set_A Size'] = int(len(set_a))
    summary_df.at[T, 'Set_B Size'] = int(len(ids_matched))
    summary_df.at[T, 'Duplicates'] = int((len(ids_matched)-len(set_a)) + (len(ids_matched)-len(set_b)))
    
summary_df

  0%|                                                                                            | 0/8 [00:00<?, ?it/s]

Arxiv samples: 4646
Semantic samples: 20000


100%|███████████████████████████████████████████████████████████████████████████████| 8/8 [12:54:11<00:00, 5806.49s/it]


,Matches,Set_A Size,Set_B Size,Duplicates
0.65,247.0,141.0,247.0,115.0
0.70,225.0,127.0,225.0,105.0
0.75,191.0,103.0,191.0,89.0
0.80,135.0,65.0,135.0,70.0
0.85,127.0,60.0,127.0,67.0
0.90,0.0,0.0,0.0,0.0
0.95,0.0,0.0,0.0,0.0
1.00,0.0,0.0,0.0,0.0


In [ ]:
# Find the lowest threshold which gives no duplicates
optimal_threshold = summary_df[summary_df['Duplicates']==0].index[0]
optimal_threshold

In [149]:
# Manually set threshold to 0.85 to trade 104 extra True Positives for 3 extra False Positive
optimal_threshold = 0.85

In [150]:
# Generate matches based on the optimal (no-duplicate) threshold
print(f'Arxiv samples: {df.shape[0]}')
print(f'GScholar samples: {df_js.shape[0]}')

# Store tuples of matches IDs, as well as singletons witouth a match
ids_matched = set()
singles_arxiv = set()
singles_gscholar = set()

# Write matches (and non-matches) to a CSV
with open('Matches_test.csv', 'w') as predictions_full:
    for block_id, arxiv_id, gscholar_id in blocks.pairwise(ds_1, ds_2):

        match , similarity = elementwise_similarity(ds_1.get_record(arxiv_id),
                                                    ds_2.get_record(gscholar_id),
                                                    match_threshold=optimal_threshold)

        if match:
            break
            ids_matched.add((arxiv_id, gscholar_id))
        else:
            singles_arxiv.add(arxiv_id)
            singles_gscholar.add(gscholar_id)
    
    # After finding all matches, write them to a csv
    for match_pair in ids_matched:
        predictions_full.write(f'{match_pair[0]},{match_pair[1]},1\n')
        # And ensure that no item in the matches is counted as a single
        try:
            singles_arxiv.remove(match_pair[0])
        except:
            pass
        try:
            singles_gscholar.remove(match_pair[1])
        except:
            pass
    
    # Then write all the singles which didn't find a match
    NULL = None
    for arxiv_id in singles_arxiv:
        predictions_full.write(f'{arxiv_id},{NULL},0\n')
    for gscholar_id in singles_gscholar:
        predictions_full.write(f'{NULL},{gscholar_id},0\n')        
        
print()
print(f'Matches: {len(ids_matched)}')
print(f'Non-Matches Arxiv: {len(singles_arxiv)}')
print(f'Non-Matches GScholar: {len(singles_gscholar)}')

Arxiv samples: 4646
GScholar samples: 20000


KeyboardInterrupt: 

### Create Merged Dataset

In [30]:
import rdflib
from rdflib import URIRef, Literal, Namespace
from rdflib.namespace import RDF, RDFS, XSD

MYNS = Namespace('http://inf558.org/myfakenamespace#')
SCHEMA = Namespace("https://schema.org/")

# Initliaze the graph
g = rdflib.Graph()

# Bind namespace and prefixes
g.bind('my_ns', MYNS)
g.bind('schema', SCHEMA)
g.bind('rdf', RDF)
g.bind('rdfs', RDFS)
g.bind('xsd', XSD)

In [32]:
# Load predictions to be used in populating the RDF
predictions_df = pd.read_csv(f'Matches.csv', header=None, names=['ARXIV_ID', 'SEMANTIC_ID', 'LABEL'])
print(f'predictions_df.shape: {predictions_df.shape}')
predicted_matches = predictions_df['LABEL'].sum()
print(f'predicted matches: {predicted_matches}  [{100*predicted_matches/predictions_df.shape[0]:.2f} %]')
predictions_df.head()

predictions_df.shape: (17227, 3)
predicted matches: 127  [0.74 %]


,ARXIV_ID,SEMANTIC_ID,LABEL
0,468,2,1
1,238,13831,1
2,268,11329,1
3,475,574,1
4,450,3042,1


In [37]:
df.keys()

Index(['index', 'ID', 'title', 'authors', 'published', 'updated', 'abstract',
       'categories', 'citations', 'arxiv_url', 'gscholar_url', 'journal'],
      dtype='object')

In [38]:
df_js.keys()

Index(['index', 'abstract', 'arxivId', 'authors', 'citationVelocity',
       'citations', 'corpusId', 'doi', 'influentialCitationCount', 'paperId',
       'references', 'title', 'topics', 'url', 'year', 'dblp_info'],
      dtype='object')

In [50]:
df_merged = pd.DataFrame(columns = ['ID', 'title', 'authors', 'published', 'updated', 
                                    'abstract', 'categories', 'citations', 'arxiv_url', 'gscholar_url',
                                    'index', 'abstract', 'arxivId', 'authors', 'citationVelocity','citations', 'corpusId', 'doi', 'influentialCitationCount', 'paperId','references', 'title', 'topics', 'url', 'year', 'dblp_info'
                                   ],
                         dtype='object')

In [52]:
ccount = 0
for i,row in predictions_df.iterrows():
    print(i,row) 
    break
for c1 in df.columns():
    df_merged.at[ccount,c1] =     df.at[468,'title']


0 ARXIV_ID       468
SEMANTIC_ID      2
LABEL            1
Name: 0, dtype: object


'Dynamic Projection Mapping onto Deforming Non-Rigid Surface Using Deformable Dot Cluster Marker'

In [174]:
x=25
s1,s2 = predictions_df.iloc[x]['ARXIV_ID'],predictions_df.iloc[x]['SEMANTIC_ID']
print(s1,s2)
ts[ts['id']==s1]['title_string'].iloc[0],ts2[ts2['id']==s2]['title_string'].iloc[0]

366 15298


('NTUA-SLP at SemEval-2018 Task 3: Tracking Ironic Tweets using Ensembles  of Word and Character Level Attentive RNNs',
 'Multi-task Self-Supervised Visual Learning')

In [183]:
for index,row in df_js.iterrows():
    if row['title'] in df['title']:
        print(row['title'])
        break

In [179]:
row['title']
df_js['title']

'Dimensional Sentiment Analysis Using a Regional CNN-LSTM Model'

# Start Code

In [13]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [14]:
import pandas as pd
import rltk
import re
from difflib import SequenceMatcher
from tqdm import tqdm
import json
import os
import pandas as pd
import rltk
import re
from difflib import SequenceMatcher
from tqdm import tqdm
import json
import os
from rltk.similarity.needleman import needleman_wunsch_score
import ast

In [2]:
os.listdir('../data')

['semantic_joined_dblp.json.zip',
 'complete_merged.pkl',
 '.DS_Store',
 'Merged_graphics.csv',
 'semantic_joined_dblp.json',
 'Merged_ai.csv',
 'output.json',
 'semantic_scholar_articles_20000.csv',
 'Merged_vision.csv',
 'Merged_nlp.csv',
 'output_compiled.json',
 'complete_merged.csv']

## Loading Semantic and DBLP JSON

In [3]:
f = open('../data/semantic_joined_dblp.json','r')
t=f.read()
f.close()
js = json.loads(t)
df_js = pd.DataFrame(js)

In [2]:
import os

In [5]:
ls = [i for i in os.listdir('../data') if "Merged" in i]

In [6]:
import pandas as pd

In [7]:
df = pd.DataFrame()
for i in ls:
    df_2 = pd.read_csv('../data/'+i)
    df = df.append(df_2)

In [8]:
len(df)

4646

## Load Data

### Naïve Data Merge

In [6]:
# How many matches can be found with a naÏve identical string approach?
d1 = df_js['title'].values
d2 = df['title'].values
print(len([1 for w in d1 if w in d2]))

159


In [7]:
def getAuthorNames(a):
    return sorted([s['name'] for s in ast.literal_eval(a)],key=lambda s:s.split(" ")[-1])

In [8]:
def getAuthorNames_2(b):
    return sorted(ast.literal_eval(b),key=lambda s:s.split(" ")[-1])

In [9]:
df_js['authors'] = list(map(getAuthorNames,df_js['authors']))

In [10]:
df['authors'] = list(map(getAuthorNames_2,df['authors']))

In [11]:
def authorComparison(s_authors,j_authors,year):
    score = []
#     thres = 9 - (2.5*year)
    thres = 6
    
    if len(s_authors)==0 and len(j_authors)==0:
        return 0 
    if len(s_authors)==len(j_authors):
        for a1,a2 in zip(s_authors,j_authors):
            score.append(needleman_wunsch_score(a1, a2, match=2, mismatch=-1, gap=-0.5, score_table=None))
#             print(score)
        if sum(score)/len(score)>thres:
            return 1
    return 0
def getPlainString(string):
    return ''.join([s for s in string if not s.isnumeric()]).strip()

In [12]:
matches = []
c = -1
df_merged = pd.DataFrame(columns = ["a_"+i for i in list(df.keys())]+["b_"+i for i in list(df_js.keys())],
                         dtype='object')

for index,row1 in df_js.iterrows():
#     print(index)
#     print(row)
    if row1['doi'] in df_merged['b_doi'].values:
        continue
    
    c=c+1
    row2 = df[df['title']==row1['title']]
    
    if len(row2)>0:
        score = authorComparison(row1['authors'],row2['authors'].iloc[0],1)
        if score==1:
            for i in row1.keys():
                df_merged.at[c,"b_"+i] = row1[i]
            for i in row2.keys():
                df_merged.at[c,"a_"+i] = row2[i].iloc[0]
            continue
    for i in row1.keys():
        
        df_merged.at[c,"b_"+i] = row1[i]
    
    
    if c%1000==0:
        print(c)
#     for i in row2.keys():
#         df_merged.at[c,"a_"+i] = row2[i]


0
1000
2000
3000
4000
5000
6000
7000
8000
9000


KeyboardInterrupt: 

In [ ]:
df_merged.keys()

In [ ]:
df_merged_dup = df_merged.copy()

In [650]:
ls = df_merged_dup['a_ID'].values
co = 0
# c=19999
for index,row in df.iterrows():
    if row['ID'] not in ls:
        co=co+1
        c = c+1
        for i in row.keys():
            df_merged_dup.at[c,"a_"+i] = row[i]


In [651]:
len(df_merged_dup)

20534

In [652]:
# df_merged_dup.to_pickle('../data/complete_merged.pkl')

In [653]:
# df_merged_dup.to_csv('../data/complete_merged.csv')

In [117]:
df_final = pd.DataFrame(columns = ['id','title','authors','yearPublished','yearUpdated','abstract','categories','journal','arxivUrl','gscholarUrl','semanticUrl','citationVelocity','influentialCitationCount','citations','references'])

### Creating DataFrame

In [118]:

# NEW_ID = 0 
c_c = 0
st = datetime.now()
for index,row in df.iterrows():
#     print(index)
#     print(row)


    # row=df.iloc[2]

    NEW_ID = index

    ### URI ###
#     node_uri = URIRef(str(NEW_ID))
#     g.add((node_uri, RDF.type, SCHEMA.ScholarlyArticle))
    df_final.at[NEW_ID,'id'] = NEW_ID
    
    if  str(row['a_title'])!="nan":
        title = row['a_title']
    else:
        title = row['b_title']
    if title:
        df_final.at[NEW_ID,'title'] = title
#         g.add((node_uri, SCHEMA.headline, Literal(title, datatype=SCHEMA.Text)))

    if str(row['a_authors'])!="nan":
        authors = row['a_authors']
    else:
        authors = row['b_authors']
    if authors:
        df_final.at[NEW_ID,'authors'] = authors
#         [g.add((node_uri, SCHEMA.author, Literal(author, datatype=SCHEMA.Person))) for author in authors]

    published = None
    if str(row['b_year'])!="nan":
        published = int(row['b_year'])
    else:
        pub =row["a_published"]
        if str(pub)!="nan":
            
            published = datetime.strptime(pub ,"%Y-%m-%dT%H:%M:%SZ").year
        
    if published:
        df_final.at[NEW_ID,'yearPublished'] = published
#         g.add((node_uri, SCHEMA.datePublished, Literal(published, datatype=SCHEMA.DateTime)))

    try:
        if str(row['a_updated'])!="nan":
            updated = datetime.strptime(row["a_updated"],"%Y-%m-%dT%H:%M:%SZ").year
            df_final.at[NEW_ID,'yearUpdated'] = updated
#             g.add((node_uri, SCHEMA.dateModified, Literal(updated, datatype=SCHEMA.DateTime)))
    except:
        pass


    try:
        if str(row['b_abstract'])!="nan":
            abstract = row['b_abstract']
        else:
            abstract = row['a_abstract']
        df_final.at[NEW_ID,'abstract'] = abstract
#         g.add((node_uri, SCHEMA.abstract, Literal(abstract, datatype=SCHEMA.Text)))
    except:
        pass


    try:
        if str(row['a_categories'])!="nan":
            categories = ast.literal_eval(row['a_categories'])
            df_final.at[NEW_ID,'categories'] = categories
#             for category in categories:
#                 g.add((node_uri, SCHEMA.genre, Literal(category, datatype=SCHEMA.Text)))
    except:
        pass
    if str(row['a_journal'])!="nan":
        journal = row['a_journal']
        df_final.at[NEW_ID,'journal'] = journal
#         g.add((node_uri, SCHEMA.publisher, Literal(journal, datatype=SCHEMA.Text))) #datatype=SCHEMA.Organisation


    if str(row['a_arxiv_url'])!="nan":
        arxiv_url = row['a_arxiv_url']
        df_final.at[NEW_ID,'arxivUrl'] = arxiv_url
#         g.add((node_uri, SCHEMA.url, Literal(arxiv_url, datatype=SCHEMA.URL)))


    if str(row['a_gscholar_url'])!="nan":
        g_scholar_url = row['a_gscholar_url']
        df_final.at[NEW_ID,'gscholarUrl'] = g_scholar_url
#         g.add((node_uri, SCHEMA.url, Literal(g_scholar_url, datatype=SCHEMA.URL)))

    if str(row['b_url'])!="nan":
        g_scholar_url = row['b_url']
        df_final.at[NEW_ID,'semanticUrl'] = g_scholar_url
#         g.add((node_uri, SCHEMA.url, Literal(g_scholar_url, datatype=SCHEMA.URL)))

    if str(row['b_citationVelocity'])!="nan":
        num = row['b_citationVelocity']
        df_final.at[NEW_ID,'citationVelocity'] = num
#         g.add((node_uri, MYNS.citationVelocity, Literal(num))) #datatype=SCHEMA.Organisation

    if str(row['b_influentialCitationCount'])!="nan":
        num = row['b_influentialCitationCount']
        df_final.at[NEW_ID,'influentialCitationCount'] = num
#         g.add((node_uri, MYNS.b_influentialCitationCount, Literal(num))) #datatype=SCHEMA.Organisation

    try:
        citations_1 = ast.literal_eval(row['b_citations'])
    except:
        citations_1 = []
    try:
        citations_2 = row['b_dblp_info']['references_opencitations']
    except:
        citations_2 = []
    citations_1.extend(citations_2)

    c_final_list = []
    
    for c in citations_1:

        if 'citing' in c:
            c_doi = c['citing'].split(" => ")[-1]
            citing_row = df[df['b_doi']==c_doi]
            if len(citing_row)>0:
                citing_id = citing_row.index[0]
                cite_uri = URIRef(str(citing_id))
                c_final_list.append(cite_uri)
#                 g.add((node_uri, SCHEMA.citation, cite_uri))
            else:
                cite_uri = c_doi
#                 len_df = len(df)
#                 cite_uri = URIRef(str(len_df))
#                 df.at[len_df,'b_doi'] = c_doi
#                 try:
#                     df.at[len_df,'b_year'] = float(c['creation'].split(" => ")[-1])
#                 except:
#                     pass
                c_final_list.append(cite_uri)
#                 g.add((node_uri, SCHEMA.citation, Literal(cite_uri)))
        else:
            try:
                c_doi = c['doi']
            except:
                c_doi = None
            if c_doi:
                citing_row = df[df['b_doi']==c_doi]
#                 if c_doi:
#                     citing_row = df[df['b_doi']==c_doi]
#                 else:
#                     citing_row = df[df['b_title']==c_title]

                if len(citing_row)>0:
                    citing_id = citing_row.index[0]
                    cite_uri = URIRef(str(citing_id))
                    c_final_list.append(cite_uri)
#                     g.add((node_uri, SCHEMA.citation, cite_uri))
                else:
                    cite_uri = c_doi
#                     len_df = len(df)
#                     cite_uri = URIRef(str(len_df))
#                     df.at[len_df,'b_doi'] = c_doi
#                     if c['arxivId']:
#                         df.at[len_df,'b_arxivId'] = c['arxivId']
#                     if c['authors']:
#                         df.at[len_df,'b_authors'] = [i['name'] for i in c['authors']]
#                     if c['title']:
#                         df.at[len_df,'b_title'] = c['title']
#                     if c['url']:
#                         df.at[len_df,'b_url'] = c['url']
#                     if c['year']:
#                         df.at[len_df,'b_year'] = c['year'] 
                    c_final_list.append(cite_uri)
#                     g.add((node_uri, SCHEMA.citation, Literal(cite_uri)))
                

    #     else:
    #         g.add((node_uri, SCHEMA.citation,Literal(c_doi, datatype=SCHEMA.Text)))

    df_final.at[NEW_ID,'citations'] = list(set(c_final_list))
    
    try:
        references_1 = ast.literal_eval(row['b_references'])
    except:
        referneces_1 = []
    try:
        references_2 = row['b_dblp_info']['references_crossref']
    except:
        references_2 = []
    references_1.extend(references_2)   

    ref_final_list = []
    
    for c in references_1:
        if 'key' in c:
            citing_row = []
            c_doi = None

            if "DOI" in c:
                c_doi = c["DOI"]
                citing_row = df[df['b_doi']==c_doi]

            elif "article-title" in c:
                c_title = c['article-title']
                citing_row = df[(df['b_title']==c_title) | (df['a_title']==c_title)]


            if len(citing_row)>0:
                citing_id = citing_row.index[0]
                cite_uri = URIRef(str(citing_id))
                ref_final_list.append(cite_uri)
#                 g.add((node_uri, MYNS.references, cite_uri))
            else:
                cite_uri = c_doi 
                
#                 len_df = len(df)
#                 cite_uri = URIRef(str(len_df))

#                 try:
#                     df.at[len_df,'b_doi'] = c_doi
#                 except:
#                     pass
#                 try:
#                     df.at[len_df,'b_year'] = float(c['year'])
#                 except:
#                     pass
#                 try:
#                     df.at[len_df,'b_author'] = c['author']
#                 except:
#                     pass
#                 try:
#                     df.at[len_df,'b_title'] = c['article-title']
#                 except:
#                     pass
                ref_final_list.append(cite_uri)
#                 g.add((node_uri, MYNS.references, Literal(cite_uri)))
        else:
            try:
                c_doi = c['doi']
            except:
                c_doi = None
                pass
            if c_doi:
                citing_row = df[df['b_doi']==c_doi]
#                 if c_doi:
#                     citing_row = df[df['b_doi']==c_doi]
#                 else:
#                     citing_row = df[df['b_title']==c_title]

                if len(citing_row)>0:
                    citing_id = citing_row.index[0]
                    cite_uri = URIRef(str(citing_id))
                    ref_final_list.append(cite_uri)
#                     g.add((node_uri, MYNS.references, cite_uri))
                else:
                    cite_uri = c_doi
#                     len_df = len(df)
#                     cite_uri = URIRef(str(len_df))
#                     df.at[len_df,'b_doi'] = c_doi
#                     if c['arxivId']:
#                         df.at[len_df,'b_arxivId'] = c['arxivId']
#                     if c['authors']:
#                         df.at[len_df,'b_authors'] = [i['name'] for i in c['authors']]
#                     if c['title']:
#                         df.at[len_df,'b_title'] = c['title']
#                     if c['url']:
#                         df.at[len_df,'b_url'] = c['url']
#                     if c['year']:
#                         df.at[len_df,'b_year'] = c['year']  
#                     g.add((node_uri, MYNS.references, Literal(cite_uri)))
                    ref_final_list.append(cite_uri)

    #     else:
    #         g.add((node_uri, SCHEMA.citation,Literal(c_doi, datatype=SCHEMA.Text)))
    df_final.at[NEW_ID,'references'] = list(set(ref_final_list))
    c_c = c_c+1
    if c_c%100==0:
        et = datetime.now()
        print((et-st).total_seconds())
        print(len(df))
        print(c_c)
        
        

11.360121
20534
100
19.281095
20534
200
36.48304
20534
300
46.033151
20534
400
63.794891
20534
500
73.962508
20534
600
81.571125
20534
700
88.012834
20534
800
100.325919
20534
900
113.99596
20534
1000
127.987083
20534
1100
135.499374
20534
1200
147.206104
20534
1300
167.818165
20534
1400
178.855763
20534
1500
186.100404
20534
1600
197.174888
20534
1700
203.21058
20534
1800
207.858666
20534
1900
215.00275
20534
2000
222.653097
20534
2100
229.621485
20534
2200
236.397039
20534
2300
243.324421
20534
2400
250.787283
20534
2500
256.62082
20534
2600
261.913084
20534
2700
271.390982
20534
2800
278.735798
20534
2900
289.24742
20534
3000
303.261372
20534
3100
314.277622
20534
3200
321.996457
20534
3300
327.676616
20534
3400
340.876159
20534
3500
349.205355
20534
3600
353.757299
20534
3700
361.214629
20534
3800
369.109771
20534
3900
379.531903
20534
4000
383.952506
20534
4100
392.724554
20534
4200
400.482516
20534
4300
410.516747
20534
4400
418.427245
20534
4500
428.553467
20534
4600
439.848592


In [121]:
# df_final.to_csv('final_dataset.csv')

In [122]:
# df_final.to_pickle('final_dataset.pkl')

In [125]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [174]:
import rdflib
from rdflib import URIRef, Literal, Namespace
from rdflib.namespace import RDF, RDFS, XSD
from datetime import datetime
import math
import ast
import pandas as pd

In [175]:
df_final = pd.read_pickle('final_dataset.pkl')
df = pd.read_pickle('../data/complete_merged.pkl')

In [182]:
all_authors = []
[all_authors.extend(i) for i in df_final['authors'] if isinstance(i,list)]
all_authors = list(set(all_authors))
author_dict = {i:c for i,c in zip(all_authors,range(1000000,1000000+len(all_authors)))}
# SCHEMA PERSON

In [183]:
all_categories = []
[all_categories.extend(i) for i in df_final['categories'] if isinstance(i,list)]
all_categories = list(set(all_categories))
category_dict = {i:c for i,c in zip(all_categories,range(2000000,2000000+len(all_categories)))}
# , datatype=SCHEMA.Text)

In [184]:
all_journals = []
all_journals = list(df_final['journal'][df_final['journal'].notnull()])
all_journals = list(map(lambda s:''.join([i for i in s if i.isalpha() or i==" "]).strip(),all_journals))
all_journals = list(set(all_journals))
journal_dict = {i:c for i,c in zip(all_journals,range(3000000,3000000+len(all_journals)))}
# , datatype=SCHEMA.Text)


In [199]:




MYNS = Namespace('http://inf558.org/myfakenamespace#')
SCHEMA = Namespace("https://schema.org/")

# Initliaze the graph
g = rdflib.Graph()

# Bind namespace and prefixes
g.bind('my_ns', MYNS)
g.bind('schema', SCHEMA)
g.bind('rdf', RDF)
g.bind('rdfs', RDFS)
g.bind('xsd', XSD)


In [200]:
doi_dict = {i:c for (i,c) in zip(df['b_doi'],range(len(df['b_doi'])))}
doi_dict_count = 20534

In [201]:
from tqdm import tqdm

In [202]:

# NEW_ID = 0 
c_c = 0
st = datetime.now()
for index,row in tqdm(df.iterrows()):
#     print(index)
#     print(row)
    if index%10000==0:
        print(len(doi_dict))
    

    # row=df.iloc[2]

    NEW_ID = index

    ### URI ###
    node_uri = URIRef(str(NEW_ID))
    g.add((node_uri, RDF.type, SCHEMA.ScholarlyArticle))
    
    if  str(row['b_doi'])!="nan":
        b_doi = row['b_doi']
        g.add((node_uri, MYNS.doi, Literal(b_doi, datatype=SCHEMA.Text)))

    
    if  str(row['a_title'])!="nan":
        title = row['a_title']
    else:
        title = row['b_title']
    if title:
        g.add((node_uri, SCHEMA.headline, Literal(title, datatype=SCHEMA.Text)))


    if str(row['a_authors'])!="nan":
        authors = row['a_authors']
    else:
        authors = row['b_authors']
    if authors:
#         [g.add((node_uri, SCHEMA.author, Literal(author, datatype=SCHEMA.Person))) for author in authors]
        for author in authors:
            g.add((node_uri, SCHEMA.author, URIRef(str(author_dict[author])))) 
            
    published = None
    if str(row['b_year'])!="nan":
        published = int(row['b_year'])
    else:
        pub =row["a_published"]
        if str(pub)!="nan":
            published = datetime.strptime(pub ,"%Y-%m-%dT%H:%M:%SZ").year
        
    if published:
        g.add((node_uri, SCHEMA.datePublished, Literal(published, datatype=SCHEMA.DateTime)))

    try:
        if str(row['a_updated'])!="nan":
            updated = datetime.strptime(row["a_updated"],"%Y-%m-%dT%H:%M:%SZ").year
            g.add((node_uri, SCHEMA.dateModified, Literal(updated, datatype=SCHEMA.DateTime)))
    except:
        pass


    try:
        if str(row['b_abstract'])!="nan":
            abstract = row['b_abstract']
        else:
            abstract = row['a_abstract']
        g.add((node_uri, SCHEMA.abstract, Literal(abstract, datatype=SCHEMA.Text)))
    except:
        pass


    try:
        if str(row['a_categories'])!="nan":
            categories = ast.literal_eval(row['a_categories'])
            for category in categories:
                g.add((node_uri, SCHEMA.genre, URIRef(str(category_dict[category]))))
    except:
        pass

    if str(row['a_journal'])!="nan":
        s = row['a_journal']
        journal = ''.join([i for i in s if i.isalpha() or i==" "]).strip()
        
        g.add((node_uri, SCHEMA.publisher, URIRef(str(journal_dict[journal])))) #datatype=SCHEMA.Organisation


    if str(row['a_arxiv_url'])!="nan":
        arxiv_url = row['a_arxiv_url']

        g.add((node_uri, SCHEMA.url, Literal(arxiv_url, datatype=SCHEMA.URL)))


    if str(row['a_gscholar_url'])!="nan":
        g_scholar_url = row['a_gscholar_url']

        g.add((node_uri, SCHEMA.url, Literal(g_scholar_url, datatype=SCHEMA.URL)))

    if str(row['b_url'])!="nan":
        g_scholar_url = row['b_url']

        g.add((node_uri, SCHEMA.url, Literal(g_scholar_url, datatype=SCHEMA.URL)))

    if str(row['b_citationVelocity'])!="nan":
        num = row['b_citationVelocity']
        g.add((node_uri, MYNS.citationVelocity, Literal(num))) #datatype=SCHEMA.Organisation

    if str(row['b_influentialCitationCount'])!="nan":
        num = row['b_influentialCitationCount']
        g.add((node_uri, MYNS.b_influentialCitationCount, Literal(num))) #datatype=SCHEMA.Organisation

    try:
        citations_1 = ast.literal_eval(row['b_citations'])
    except:
        citations_1 = []
    try:
        citations_2 = row['b_dblp_info']['references_opencitations']
    except:
        citations_2 = []
    citations_1.extend(citations_2)

    for c in citations_1:

        if 'citing' in c:
            c_doi = c['citing'].split(" => ")[-1]
#             citing_row = df[df['b_doi']==c_doi]
            citing_id=-1
            try:
                citing_id = doi_dict[c_doi]
            except:
                pass
            
            if citing_id>-1:
#                 citing_id = citing_row.index[0]
                cite_uri = URIRef(str(citing_id))
                g.add((node_uri, SCHEMA.citation, cite_uri))
            else:
#                 len_df = len(df)
                cite_uri = URIRef(str(doi_dict_count))
                doi_dict[c_doi] = doi_dict_count
                doi_dict_count+=1

#                 df.at[len_df,'b_doi'] = c_doi
#                 try:
#                     df.at[len_df,'b_year'] = float(c['creation'].split(" => ")[-1])
#                 except:
#                     pass
                
                g.add((node_uri, SCHEMA.citation, cite_uri))
        else:
            try:
                c_doi = c['doi']
            except:
                c_doi = None
            if c_doi:
                citing_id=-1
                try:
                    citing_id = doi_dict[c_doi]
                except:
                    pass
            #                 if c_doi:
#                     citing_row = df[df['b_doi']==c_doi]
#                 else:
#                     citing_row = df[df['b_title']==c_title]

                if citing_id>-1:
#                     citing_id = citing_row.index[0]
                    cite_uri = URIRef(str(citing_id))
                    g.add((node_uri, SCHEMA.citation, cite_uri))
                else:
                    cite_uri = URIRef(str(doi_dict_count))
                    doi_dict[c_doi] = doi_dict_count
                    doi_dict_count+=1
                    #                     len_df = len(df)
#                     cite_uri = URIRef(str(len_df))
#                     df.at[len_df,'b_doi'] = c_doi
#                     if c['arxivId']:
#                         df.at[len_df,'b_arxivId'] = c['arxivId']
#                     if c['authors']:
#                         df.at[len_df,'b_authors'] = [i['name'] for i in c['authors']]
#                     if c['title']:
#                         df.at[len_df,'b_title'] = c['title']
#                     if c['url']:
#                         df.at[len_df,'b_url'] = c['url']
#                     if c['year']:
#                         df.at[len_df,'b_year'] = c['year']            
                    g.add((node_uri, SCHEMA.citation, cite_uri))
                

    #     else:
    #         g.add((node_uri, SCHEMA.citation,Literal(c_doi, datatype=SCHEMA.Text)))

    try:
        references_1 = ast.literal_eval(row['b_references'])
    except:
        referneces_1 = []
    try:
        references_2 = row['b_dblp_info']['references_crossref']
    except:
        references_2 = []
    references_1.extend(references_2)   

    for c in references_1:
        if 'key' in c:
            citing_row = []
            c_doi = None

            if "DOI" in c:
                c_doi = c["DOI"]
#                 citing_row = df[df['b_doi']==c_doi]

#             elif "article-title" in c:
#                 c_title = c['article-title']
#                 citing_row = df[(df['b_title']==c_title) | (df['a_title']==c_title)]
            if c_doi:
                citing_id=-1
                try:
                    citing_id = doi_dict[c_doi]
                except:
                    pass

                if citing_id>-1:
#                     citing_id = citing_row.index[0]
                    cite_uri = URIRef(str(citing_id))
                    g.add((node_uri, MYNS.references, cite_uri))
                else:
                    cite_uri = URIRef(str(doi_dict_count))
                    doi_dict[c_doi] = doi_dict_count
                    doi_dict_count+=1
    #                 len_df = len(df)
    #                 cite_uri = URIRef(str(len_df))

    #                 try:
    #                     df.at[len_df,'b_doi'] = c_doi
    #                 except:
    #                     pass
    #                 try:
    #                     df.at[len_df,'b_year'] = float(c['year'])
    #                 except:
    #                     pass
    #                 try:
    #                     df.at[len_df,'b_author'] = c['author']
    #                 except:
    #                     pass
    #                 try:
    #                     df.at[len_df,'b_title'] = c['article-title']
    #                 except:
    #                     pass

                    g.add((node_uri, MYNS.references, cite_uri))
        else:
            try:
                c_doi = c['doi']
            except:
                c_doi = None
                pass
            if c_doi:
                citing_id=-1
                try:
                    citing_id = doi_dict[c_doi]
                except:
                    pass#                 if c_doi:
#                     citing_row = df[df['b_doi']==c_doi]
#                 else:
#                     citing_row = df[df['b_title']==c_title]

                if citing_id>-1:
#                     citing_id = citing_row.index[0]
                    cite_uri = URIRef(str(citing_id))
                    g.add((node_uri, MYNS.references, cite_uri))
                else:
                    cite_uri = URIRef(str(doi_dict_count))
                    doi_dict[c_doi] = doi_dict_count
                    doi_dict_count+=1
                    #                     len_df = len(df)
#                     cite_uri = URIRef(str(len_df))
#                     df.at[len_df,'b_doi'] = c_doi
#                     if c['arxivId']:
#                         df.at[len_df,'b_arxivId'] = c['arxivId']
#                     if c['authors']:
#                         df.at[len_df,'b_authors'] = [i['name'] for i in c['authors']]
#                     if c['title']:
#                         df.at[len_df,'b_title'] = c['title']
#                     if c['url']:
#                         df.at[len_df,'b_url'] = c['url']
#                     if c['year']:
#                         df.at[len_df,'b_year'] = c['year']  
                    g.add((node_uri, MYNS.references, cite_uri))

    #     else:
    #         g.add((node_uri, SCHEMA.citation,Literal(c_doi, datatype=SCHEMA.Text)))
#     c_c = c_c+1
#     if c_c%100==0:
#         et = datetime.now()
#         print((et-st).total_seconds())
#         print(len(df))
#         print(c_c)
        

23it [00:00, 124.83it/s]

20534


10010it [02:14, 103.49it/s]

356206


20087it [03:31, 572.14it/s]

453463


20534it [03:32, 96.71it/s] 


In [212]:
len(doi_dict)

453463

In [216]:
doi_dict_inverse = {i:c for c,i in zip(doi_dict,range(0,len(doi_dict))) if i > 20534}

In [221]:
for d in tqdm(doi_dict_inverse):
    node_uri = URIRef(str(d))
    g.add((node_uri, RDF.type, SCHEMA.ScholarlyArticle))
    b_doi = doi_dict_inverse[d]
    g.add((node_uri, MYNS.doi, Literal(b_doi, datatype=SCHEMA.Text)))

100%|██████████| 432928/432928 [00:21<00:00, 19848.57it/s]


In [231]:
for d in tqdm(author_dict):
    node_uri = URIRef(str(author_dict[d]))
    g.add((node_uri, RDF.type, SCHEMA.Person))
    g.add((node_uri, SCHEMA.name, Literal(d, datatype=SCHEMA.Text)))

100%|██████████| 50677/50677 [00:03<00:00, 14960.68it/s]


In [233]:
for d in tqdm(journal_dict):
    node_uri = URIRef(str(journal_dict[d]))
    g.add((node_uri, RDF.type, SCHEMA.Publisher))
    g.add((node_uri, SCHEMA.name, Literal(d, datatype=SCHEMA.Text)))

100%|██████████| 86/86 [00:00<00:00, 6250.93it/s]


In [236]:
for d in tqdm(category_dict):
    node_uri = URIRef(str(category_dict[d]))
    g.add((node_uri, RDF.type, SCHEMA.genre))
    g.add((node_uri, SCHEMA.name, Literal(d, datatype=SCHEMA.Text)))

100%|██████████| 207/207 [00:00<00:00, 12791.47it/s]


In [237]:
# print(g.serialize(format="turtle").decode()[0:10000])

In [70]:
# author_splits = {}
# for i in all_authors:
#     sp = i.split(" ")[-1]
#     if sp not in author_splits:
#         author_splits[sp] = [i]
#     else:
#         author_splits[sp].append(i)

In [71]:
# author_dict = {}
# for i in tqdm(all_authors):
# #     print(i)
#     if i not in author_dict:
#         sp = i.split(" ")[-1]
#         for j in author_splits[sp]:
#             score = needleman_wunsch_score(i, j, match=2, mismatch=-1, gap=-0.5, score_table=None)
#             if score>20:
#                 author_dict[i] = j
#                 break


100%|██████████| 50677/50677 [03:46<00:00, 223.66it/s]


In [72]:
len(author_dict)

29411

In [83]:

try:
    citations_1 = ast.literal_eval(row['b_citations'])
except:
    citations_1 = []
try:
    citations_2 = row['b_dblp_info']['references_opencitations']
except:
    citations_2 = []

In [91]:
len(df.iloc[0]['b_dblp_info']['references_opencitations'])

105

In [1082]:
df.iloc[1]["b_dblp_info"]

nan

In [1080]:
len(df.iloc[1]["dblp_info"])

KeyError: 'dblp_info'

In [1066]:
# a_journal
# # b_arxivId
# b_citationVelocity
# b_influentialCitationCount
# b_topics
# b_url - semantic url 

In [1049]:
print(len(citations_1))
print(len(citations_2))

417
108


In [1053]:
len(references_1)

87

@prefix my_ns: <http://inf558.org/myfakenamespace#> .
@prefix schema: <https://schema.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<0> a schema:ScholarlyArticle ;
    my_ns:b_influentialCitationCount 52 ;
    my_ns:citationVelocity 150 ;
    my_ns:references <0>,
        <20580>,
        <20581>,
        <20582>,
        <20583>,
        <20584>,
        <20585>,
        <20586>,
        <20587>,
        <20588>,
        <20589>,
        <20590>,
        <20591>,
        <20592>,
        <20593>,
        <20594>,
        <20595>,
        <20596>,
        <20597>,
        <20598>,
        <20599>,
        <20600>,
        <20601>,
        <20602>,
        <20603>,
        <20604>,
        <20605>,
        <20606>,
        <20607>,
        <20608>,
        <20609>,
        <20610>,
        <20611>,
        <20612>,
        <20613>,
        <20614>,
        <20615>,
        <20616>,
        <20617>,
        <20618>,
        <20619>,
        <20620>,
        <20621>,
       

In [ ]:
row = df.iloc[2]

In [1030]:
df.iloc[2]['a_journal']

'Proceedings of the IEEE International Conference on Computer Vision, 37-45'

In [ ]:
if str(row['a_journal'])!="nan":
    journal = row['a_journal']

    [g.add((node_uri, SCHEMA.genre, Literal(category, datatype=SCHEMA.Text))) for category in categories]

In [1013]:
df.iloc[347]['b_doi']

'10.1007/s41745-019-0099-3'

In [1019]:
for i in citations_1:
    if (i['doi']==df.iloc[347]['b_doi']):
        break

In [1020]:
i['doi']

'10.1007/s41745-019-0099-3'

20534

In [881]:
citations_1[0]['doi']

'10.1007/978-3-030-01246-5_18'

In [587]:
df['b_doi']

0        10.1007/978-3-642-21735-7_6
1        10.1109/ACCESS.2020.2982782
2               10.1109/ICCV.2015.13
3            10.1109/CVPR.2018.00830
4           10.1109/TIP.2020.3014608
                    ...             
24400                            NaN
24401                            NaN
24402                            NaN
24403                            NaN
24404                            NaN
Name: b_doi, Length: 24405, dtype: object

In [588]:
df[df['b_doi']==citations_2[99]['citing'].split(" => ")[-1]]

,a_index,a_ID,a_title,a_authors,a_published,a_updated,a_abstract,a_categories,a_citations,a_arxiv_url,...,b_corpusId,b_doi,b_influentialCitationCount,b_paperId,b_references,b_title,b_topics,b_url,b_year,b_dblp_info
526,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7469546,10.1007/978-3-319-46475-6_1,3,d93ececdc44e7700cd84fc75ca069125022d8c9d,"[{'arxivId': '1312.6114', 'authors': [{'author...",The Curious Robot: Learning Visual Representat...,"[{'topic': 'Fundamental interaction', 'topicId...",https://www.semanticscholar.org/paper/d93ececd...,2016,"{'@score': '9', '@id': '1695087', 'info': {'au..."
11347,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7469546,10.1007/978-3-319-46475-6_1,3,d93ececdc44e7700cd84fc75ca069125022d8c9d,"[{'arxivId': '1312.6114', 'authors': [{'author...",The Curious Robot: Learning Visual Representat...,"[{'topic': 'Fundamental interaction', 'topicId...",https://www.semanticscholar.org/paper/d93ececd...,2016,"{'@score': '9', '@id': '1696642', 'info': {'au..."
18299,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7469546,10.1007/978-3-319-46475-6_1,3,d93ececdc44e7700cd84fc75ca069125022d8c9d,"[{'arxivId': None, 'authors': [{'authorId': '1...",The Curious Robot: Learning Visual Representat...,"[{'topic': 'Fundamental interaction', 'topicId...",https://www.semanticscholar.org/paper/d93ececd...,2016,"{'@score': '9', '@id': '1696642', 'info': {'au..."


In [599]:
sum(df['b_doi'].value_counts()>1)

2281

ValueError: Cannot mask with non-boolean array containing NA / NaN values

,a_index,a_ID,a_title,a_authors,a_published,a_updated,a_abstract,a_categories,a_citations,a_arxiv_url,...,b_corpusId,b_doi,b_influentialCitationCount,b_paperId,b_references,b_title,b_topics,b_url,b_year,b_dblp_info
414,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,62841734,10.1109/tpami.2020.2992393,11,4c94ee7df6bc2bfcac76703be4f059a79010f7e5,"[{'arxivId': '1803.02276', 'authors': [{'autho...",Self-supervised Visual Feature Learning with D...,"[{'topic': 'Feature learning', 'topicId': '205...",https://www.semanticscholar.org/paper/4c94ee7d...,2020,"{'@score': '11', '@id': '793611', 'info': {'au..."
724,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,62841734,10.1109/tpami.2020.2992393,11,4c94ee7df6bc2bfcac76703be4f059a79010f7e5,"[{'arxivId': '1803.02276', 'authors': [{'autho...",Self-supervised Visual Feature Learning with D...,"[{'topic': 'Feature learning', 'topicId': '205...",https://www.semanticscholar.org/paper/4c94ee7d...,2020,"{'@score': '11', '@id': '793611', 'info': {'au..."
11360,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,62841734,10.1109/tpami.2020.2992393,11,4c94ee7df6bc2bfcac76703be4f059a79010f7e5,"[{'arxivId': '1803.02276', 'authors': [{'autho...",Self-supervised Visual Feature Learning with D...,"[{'topic': 'Feature learning', 'topicId': '205...",https://www.semanticscholar.org/paper/4c94ee7d...,2020,"{'@score': '11', '@id': '795166', 'info': {'au..."
13358,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,62841734,10.1109/tpami.2020.2992393,11,4c94ee7df6bc2bfcac76703be4f059a79010f7e5,"[{'arxivId': '1803.02276', 'authors': [{'autho...",Self-supervised Visual Feature Learning with D...,"[{'topic': 'Feature learning', 'topicId': '205...",https://www.semanticscholar.org/paper/4c94ee7d...,2020,"{'@score': '11', '@id': '795166', 'info': {'au..."
13388,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,62841734,10.1109/tpami.2020.2992393,11,4c94ee7df6bc2bfcac76703be4f059a79010f7e5,"[{'arxivId': '1803.02276', 'authors': [{'autho...",Self-supervised Visual Feature Learning with D...,"[{'topic': 'Feature learning', 'topicId': '205...",https://www.semanticscholar.org/paper/4c94ee7d...,2020,"{'@score': '11', '@id': '795166', 'info': {'au..."
13558,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,62841734,10.1109/tpami.2020.2992393,11,4c94ee7df6bc2bfcac76703be4f059a79010f7e5,"[{'arxivId': '1803.02276', 'authors': [{'autho...",Self-supervised Visual Feature Learning with D...,"[{'topic': 'Feature learning', 'topicId': '205...",https://www.semanticscholar.org/paper/4c94ee7d...,2020,"{'@score': '11', '@id': '795166', 'info': {'au..."
13761,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,62841734,10.1109/tpami.2020.2992393,11,4c94ee7df6bc2bfcac76703be4f059a79010f7e5,"[{'arxivId': '1803.02276', 'authors': [{'autho...",Self-supervised Visual Feature Learning with D...,"[{'topic': 'Feature learning', 'topicId': '205...",https://www.semanticscholar.org/paper/4c94ee7d...,2020,"{'@score': '11', '@id': '795166', 'info': {'au..."
13973,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,62841734,10.1109/tpami.2020.2992393,11,4c94ee7df6bc2bfcac76703be4f059a79010f7e5,"[{'arxivId': '1803.02276', 'authors': [{'autho...",Self-supervised Visual Feature Learning with D...,"[{'topic': 'Feature learning', 'topicId': '205...",https://www.semanticscholar.org/paper/4c94ee7d...,2020,"{'@score': '11', '@id': '795166', 'info': {'au..."
14571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,62841734,10.1109/tpami.2020.2992393,11,4c94ee7df6bc2bfcac76703be4f059a79010f7e5,"[{'arxivId': '1803.02276', 'authors': [{'autho...",Self-supervised Visual Feature Learning with D...,"[{'topic': 'Feature learning', 'topicId': '205...",https://www.semanticscholar.org/paper/4c94ee7d...,2020,"{'@score': '11', '@id': '795166', 'info': {'au..."
14907,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,62841734,10.1109/tpami.2020.2992393,11,4c94ee7df6bc2bfcac76703be4f059a79010f7e5,"[{'arxivId': '1803.02276', 'authors': [{'autho...",Self-supervised Visual Feature Learning with D...,"[{'topic': 'Feature learning', 'topic

In [595]:
doi_ls = df['b_doi'].dropna()

TypeError: cannot convert the series to <class 'float'>

In [591]:
len(df['b_doi'])

24405

In [565]:
try:
    references_1 = ast.literal_eval(df.iloc[ic]['b_references'])
except:
    references_1 = []
try:
    references_2 = df.iloc[ic]['b_dblp_info']['references_crossref']
except:
    references_2 = []
print(len(references_1))
print(len(references_2))

28
0


39
39


In [519]:
sorted(list(df.keys()),key=lambda s:s.split("_")[-1])

['a_ID',
 'a_abstract',
 'b_abstract',
 'b_arxivId',
 'a_authors',
 'b_authors',
 'a_categories',
 'b_citationVelocity',
 'a_citations',
 'b_citations',
 'b_corpusId',
 'b_doi',
 'a_index',
 'b_index',
 'b_influentialCitationCount',
 'b_dblp_info',
 'a_journal',
 'b_paperId',
 'a_published',
 'b_references',
 'a_title',
 'b_title',
 'b_topics',
 'a_updated',
 'a_arxiv_url',
 'a_gscholar_url',
 'b_url',
 'b_year']

In [518]:
print(g.serialize(format='turtle').decode())

@prefix schema: <https://schema.org/> .

<0> a schema:ScholarlyArticle ;
    schema:abstract "The artificial neural networks that are used to recognize shapes typically use one or more layers of learned feature detectors that produce scalar outputs. By contrast, the computer vision community uses complicated, hand-engineered features, like SIFT [6], that produce a whole vector of outputs including an explicit representation of the pose of the feature. We show how neural networks can be used to learn features that output a whole vector of instantiation parameters and we argue that this is a much more promising way of dealing with variations in position, orientation, scale and lighting than the methods currently employed in the neural networks community. It is also more promising than the hand-engineered features currently used in computer vision because it provides an efficient way of adapting the features to the domain."^^schema:Text ;
    schema:author "A. Krizhevsky"^^schema:Person,


In [433]:

### Author(s) ###
try:
    author_arxiv = df_arxiv[df_arxiv['ID'] == str(row['ARXIV_ID'])]['authors'].values[0]
    author_arxiv = [name.strip() for name in author_arxiv if name != '<___>']
except:
    author_arxiv = '<___>'
try:
    author_gscholar = df_gscholar[df_gscholar['ID'] == str(row['GSCHOLAR_ID'])]['authors'].values[0]
    author_gscholar = [name.strip() for name in author_gscholar if name != '<___>']
except:
    author_gscholar = '<___>'
if author_arxiv != '<___>':
    authors = list(set(author_arxiv))
else:
    authors = list(set(author_gscholar))           
[g.add((node_uri, SCHEMA.author, Literal(author, datatype=SCHEMA.Person))) for author in authors]
df_merged.at[NEW_ID, 'authors'] = authors
json_merged[NEW_ID]['authors'] = authors

### Published ###
try:
    published = str(df_arxiv[df_arxiv['ID'] == str(row['ARXIV_ID'])]['published'].values[0])
    g.add((node_uri, SCHEMA.datePublished, Literal(published, datatype=SCHEMA.DateTime)))
    df_merged.at[NEW_ID, 'published'] = published
    json_merged[NEW_ID]['published'] = published
except:
    pass


### Updated ###
try:
    updated = str(df_arxiv[df_arxiv['ID'] == str(row['ARXIV_ID'])]['updated'].values[0])
    g.add((node_uri, SCHEMA.dateModified, Literal(updated, datatype=SCHEMA.DateTime)))
    df_merged.at[NEW_ID, 'updated'] = updated
    json_merged[NEW_ID]['updated'] = updated
except:
    pass


### Abstract ###
try:
    abstract = str(df_arxiv[df_arxiv['ID'] == str(row['ARXIV_ID'])]['summary'].values[0]).strip()
    g.add((node_uri, SCHEMA.abstract, Literal(abstract, datatype=SCHEMA.Text)))
    df_merged.at[NEW_ID, 'abstract'] = abstract
    json_merged[NEW_ID]['abstract'] = abstract
except:
    pass


### Categories ###
try:
    categories = df_arxiv[df_arxiv['ID'] == str(row['ARXIV_ID'])]['categories'].values[0]
    categories = [name.strip() for name in categories if name != '<___>']
    [g.add((node_uri, SCHEMA.genre, Literal(category, datatype=SCHEMA.Text))) for category in categories]
    df_merged.at[NEW_ID, 'categories'] = categories
    json_merged[NEW_ID]['categories'] = categories
except:
    pass


### Journal ###
try:
    journal = str(df_gscholar[df_gscholar['ID'] == str(row['GSCHOLAR_ID'])]['journal'].values[0])
    g.add((node_uri, SCHEMA.publisher, Literal(journal, datatype=SCHEMA.Periodical))) #datatype=SCHEMA.Organisation
    df_merged.at[NEW_ID, 'journal'] = journal
    json_merged[NEW_ID]['journal'] = journal
except:
    pass


### Citations ###
try:
    citations = str(df_gscholar[df_gscholar['ID'] == str(row['GSCHOLAR_ID'])]['citations'].values[0])
    g.add((node_uri, SCHEMA.commentCount, Literal(citations, datatype=SCHEMA.Integer)))
    df_merged.at[NEW_ID, 'citations'] = citations
    json_merged[NEW_ID]['citations'] = citations
except:
    pass


### Arxiv URL ###
try:
    arxiv_url = str(df_arxiv[df_arxiv['ID'] == str(row['ARXIV_ID'])]['url'].values[0])
    g.add((node_uri, SCHEMA.url, Literal(arxiv_url, datatype=SCHEMA.URL)))
    df_merged.at[NEW_ID, 'arxiv_url'] = arxiv_url    
    json_merged[NEW_ID]['arxiv_url'] = arxiv_url
except:
    pass


### Google Scholar URL ###
try:
    gscholar_url = str(df_gscholar[df_gscholar['ID'] == str(row['GSCHOLAR_ID'])]['url'].values[0])
    g.add((node_uri, SCHEMA.url, Literal(gscholar_url, datatype=SCHEMA.URL)))
    df_merged.at[NEW_ID, 'gscholar_url'] = gscholar_url   
    json_merged[NEW_ID]['gscholar_url'] = gscholar_url
except:
    pass


NEW_ID += 1

# Save to disk using turtle format
g.serialize(f'Triples_{TOPIC}.ttl.', format="turtle")

# And save the merged DataFrame as CSV
df_merged.to_csv(f'Merged_{TOPIC}.csv', index=False)

# Also save as Json, just because
with open(f'Json_{TOPIC}.json', 'w') as fout:
json.dump(json_merged, fout)

rdflib.term.URIRef('0')

['a_ID',
 'a_abstract',
 'b_abstract',
 'b_arxivId',
 'a_authors',
 'b_authors',
 'a_categories',
 'b_citationVelocity',
 'a_citations',
 'b_citations',
 'b_corpusId',
 'b_doi',
 'a_index',
 'b_index',
 'b_influentialCitationCount',
 'b_dblp_info',
 'a_journal',
 'b_paperId',
 'a_published',
 'b_references',
 'a_title',
 'b_title',
 'b_topics',
 'a_updated',
 'a_arxiv_url',
 'a_gscholar_url',
 'b_url',
 'b_year']

In [420]:
math.isnan(row['a_index'])

True

In [417]:
.isnan()

AttributeError: 'float' object has no attribute 'isnan'

In [434]:
g.add((node_uri, RDF.type, SCHEMA.ScholarlyArticle))


<Graph identifier=N0fb1bac5551443b6b8085f491c0eb81f (<class 'rdflib.graph.Graph'>)>

In [ ]:
# Dataframe to store the merged datasets
df_merged = pd.DataFrame(columns = ['ID', 'title', 'authors', 'published', 'updated', 
                                    'abstract', 'categories', 'citations', 'arxiv_url', 'gscholar_url'],
                         dtype='object')
json_merged = {}

NEW_ID = 0

# Populate the RDF with predictions with a positive (1) label
for idx, row in tqdm(predictions_df.iterrows(), total=predictions_df.shape[0]):
    
    # Populate the json object
    json_merged[NEW_ID] = {}
    
    ### URI ###
    node_uri = URIRef(str(NEW_ID))
    g.add((node_uri, RDF.type, SCHEMA.ScholarlyArticle))
    df_merged.at[NEW_ID, 'ID'] = NEW_ID
    json_merged[NEW_ID]['ID'] = NEW_ID

    
    ### Title ###
    try:
        title_arxiv = str(df_arxiv[df_arxiv['ID'] == str(row['ARXIV_ID'])]['title'].values[0])
    except:
        title_arxiv = '<___>'
    try:
        title_gscholar = str(df_gscholar[df_gscholar['ID'] == str(row['GSCHOLAR_ID'])]['title'].values[0])
    except:
        title_gscholar = '<___>'
    title = title_arxiv if title_arxiv != '<___>' else title_gscholar if title_gscholar != '<___>' else None
    g.add((node_uri, SCHEMA.headline, Literal(title, datatype=SCHEMA.Text)))
    df_merged.at[NEW_ID, 'title'] = title
    json_merged[NEW_ID]['title'] = title

    
    ### Author(s) ###
    try:
        author_arxiv = df_arxiv[df_arxiv['ID'] == str(row['ARXIV_ID'])]['authors'].values[0]
        author_arxiv = [name.strip() for name in author_arxiv if name != '<___>']
    except:
        author_arxiv = '<___>'
    try:
        author_gscholar = df_gscholar[df_gscholar['ID'] == str(row['GSCHOLAR_ID'])]['authors'].values[0]
        author_gscholar = [name.strip() for name in author_gscholar if name != '<___>']
    except:
        author_gscholar = '<___>'
    if author_arxiv != '<___>':
        authors = list(set(author_arxiv))
    else:
        authors = list(set(author_gscholar))           
    [g.add((node_uri, SCHEMA.author, Literal(author, datatype=SCHEMA.Person))) for author in authors]
    df_merged.at[NEW_ID, 'authors'] = authors
    json_merged[NEW_ID]['authors'] = authors
                       
    ### Published ###
    try:
        published = str(df_arxiv[df_arxiv['ID'] == str(row['ARXIV_ID'])]['published'].values[0])
        g.add((node_uri, SCHEMA.datePublished, Literal(published, datatype=SCHEMA.DateTime)))
        df_merged.at[NEW_ID, 'published'] = published
        json_merged[NEW_ID]['published'] = published
    except:
        pass
        
                       
    ### Updated ###
    try:
        updated = str(df_arxiv[df_arxiv['ID'] == str(row['ARXIV_ID'])]['updated'].values[0])
        g.add((node_uri, SCHEMA.dateModified, Literal(updated, datatype=SCHEMA.DateTime)))
        df_merged.at[NEW_ID, 'updated'] = updated
        json_merged[NEW_ID]['updated'] = updated
    except:
        pass
          
                       
    ### Abstract ###
    try:
        abstract = str(df_arxiv[df_arxiv['ID'] == str(row['ARXIV_ID'])]['summary'].values[0]).strip()
        g.add((node_uri, SCHEMA.abstract, Literal(abstract, datatype=SCHEMA.Text)))
        df_merged.at[NEW_ID, 'abstract'] = abstract
        json_merged[NEW_ID]['abstract'] = abstract
    except:
        pass
       
                       
    ### Categories ###
    try:
        categories = df_arxiv[df_arxiv['ID'] == str(row['ARXIV_ID'])]['categories'].values[0]
        categories = [name.strip() for name in categories if name != '<___>']
        [g.add((node_uri, SCHEMA.genre, Literal(category, datatype=SCHEMA.Text))) for category in categories]
        df_merged.at[NEW_ID, 'categories'] = categories
        json_merged[NEW_ID]['categories'] = categories
    except:
        pass
          
                       
    ### Journal ###
    try:
        journal = str(df_gscholar[df_gscholar['ID'] == str(row['GSCHOLAR_ID'])]['journal'].values[0])
        g.add((node_uri, SCHEMA.publisher, Literal(journal, datatype=SCHEMA.Periodical))) #datatype=SCHEMA.Organisation
        df_merged.at[NEW_ID, 'journal'] = journal
        json_merged[NEW_ID]['journal'] = journal
    except:
        pass
     
                       
    ### Citations ###
    try:
        citations = str(df_gscholar[df_gscholar['ID'] == str(row['GSCHOLAR_ID'])]['citations'].values[0])
        g.add((node_uri, SCHEMA.commentCount, Literal(citations, datatype=SCHEMA.Integer)))
        df_merged.at[NEW_ID, 'citations'] = citations
        json_merged[NEW_ID]['citations'] = citations
    except:
        pass
            
                       
    ### Arxiv URL ###
    try:
        arxiv_url = str(df_arxiv[df_arxiv['ID'] == str(row['ARXIV_ID'])]['url'].values[0])
        g.add((node_uri, SCHEMA.url, Literal(arxiv_url, datatype=SCHEMA.URL)))
        df_merged.at[NEW_ID, 'arxiv_url'] = arxiv_url    
        json_merged[NEW_ID]['arxiv_url'] = arxiv_url
    except:
        pass
        
                       
    ### Google Scholar URL ###
    try:
        gscholar_url = str(df_gscholar[df_gscholar['ID'] == str(row['GSCHOLAR_ID'])]['url'].values[0])
        g.add((node_uri, SCHEMA.url, Literal(gscholar_url, datatype=SCHEMA.URL)))
        df_merged.at[NEW_ID, 'gscholar_url'] = gscholar_url   
        json_merged[NEW_ID]['gscholar_url'] = gscholar_url
    except:
        pass
             
                       
    NEW_ID += 1
    
# Save to disk using turtle format
g.serialize(f'Triples_{TOPIC}.ttl.', format="turtle")

# And save the merged DataFrame as CSV
df_merged.to_csv(f'Merged_{TOPIC}.csv', index=False)

# Also save as Json, just because
with open(f'Json_{TOPIC}.json', 'w') as fout:
    json.dump(json_merged, fout)

In [ ]:
df_merged.head(3)

In [ ]:
df_merged.info()

In [ ]:
json_merged

## End

In [ ]:
for index,row in df:

In [609]:
len(set(df['b_doi']))

20534

In [602]:
test_df = pd.read_csv('../data/semantic_scholar_articles_20000.csv')

In [ ]:
len(set(test_df['doi']))

In [607]:
len(test_df.drop_duplicates())

19660

In [605]:
test_df[test_df['doi']=="10.1109/tpami.2020.2992393"]

,abstract,arxivId,authors,citationVelocity,citations,corpusId,doi,influentialCitationCount,paperId,references,title,topics,url,year
414,Large-scale labeled data are generally require...,1902.06162,"[{'authorId': '29724362', 'name': 'Longlong Ji...",61,"[{'arxivId': None, 'authors': [{'authorId': '4...",62841734,10.1109/tpami.2020.2992393,11,4c94ee7df6bc2bfcac76703be4f059a79010f7e5,"[{'arxivId': '1803.02276', 'authors': [{'autho...",Self-supervised Visual Feature Learning with D...,"[{'topic': 'Feature learning', 'topicId': '205...",https://www.semanticscholar.org/paper/4c94ee7d...,2020.0
724,Large-scale labeled data are generally require...,1902.06162,"[{'authorId': '29724362', 'name': 'Longlong Ji...",61,"[{'arxivId': None, 'authors': [{'authorId': '4...",62841734,10.1109/tpami.2020.2992393,11,4c94ee7df6bc2bfcac76703be4f059a79010f7e5,"[{'arxivId': '1803.02276', 'authors': [{'autho...",Self-supervised Visual Feature Learning with D...,"[{'topic': 'Feature learning', 'topicId': '205...",https://www.semanticscholar.org/paper/4c94ee7d...,2020.0
11360,Large-scale labeled data are generally require...,1902.06162,"[{'authorId': '29724362', 'name': 'Longlong Ji...",61,"[{'arxivId': None, 'authors': [{'authorId': '4...",62841734,10.1109/tpami.2020.2992393,11,4c94ee7df6bc2bfcac76703be4f059a79010f7e5,"[{'arxivId': '1803.02276', 'authors': [{'autho...",Self-supervised Visual Feature Learning with D...,"[{'topic': 'Feature learning', 'topicId': '205...",https://www.semanticscholar.org/paper/4c94ee7d...,2020.0
13358,Large-scale labeled data are generally require...,1902.06162,"[{'authorId': '29724362', 'name': 'Longlong Ji...",61,"[{'arxivId': None, 'authors': [{'authorId': '4...",62841734,10.1109/tpami.2020.2992393,11,4c94ee7df6bc2bfcac76703be4f059a79010f7e5,"[{'arxivId': '1803.02276', 'authors': [{'autho...",Self-supervised Visual Feature Learning with D...,"[{'topic': 'Feature learning', 'topicId': '205...",https://www.semanticscholar.org/paper/4c94ee7d...,2020.0
13388,Large-scale labeled data are generally require...,1902.06162,"[{'authorId': '29724362', 'name': 'Longlong Ji...",61,"[{'arxivId': None, 'authors': [{'authorId': '4...",62841734,10.1109/tpami.2020.2992393,11,4c94ee7df6bc2bfcac76703be4f059a79010f7e5,"[{'arxivId': '1803.02276', 'authors': [{'autho...",Self-supervised Visual Feature Learning with D...,"[{'topic': 'Feature learning', 'topicId': '205...",https://www.semanticscholar.org/paper/4c94ee7d...,2020.0
13558,Large-scale labeled data are generally require...,1902.06162,"[{'authorId': '29724362', 'name': 'Longlong Ji...",61,"[{'arxivId': None, 'authors': [{'authorId': '4...",62841734,10.1109/tpami.2020.2992393,11,4c94ee7df6bc2bfcac76703be4f059a79010f7e5,"[{'arxivId': '1803.02276', 'authors': [{'autho...",Self-supervised Visual Feature Learning with D...,"[{'topic': 'Feature learning', 'topicId': '205...",https://www.semanticscholar.org/paper/4c94ee7d...,2020.0
13761,Large-scale labeled data are generally require...,1902.06162,"[{'authorId': '29724362', 'name': 'Longlong Ji...",61,"[{'arxivId': None, 'authors': [{'authorId': '4...",62841734,10.1109/tpami.2020.2992393,11,4c94ee7df6bc2bfcac76703be4f059a79010f7e5,"[{'arxivId': '1803.02276', 'authors': [{'autho...",Self-supervised Visual Feature Learning with D...,"[{'topic': 'Feature learning', 'topicId': '205...",https://www.semanticscholar.org/paper/4c94ee7d...,2020.0
13973,Large-scale labeled data are generally require...,1902.06162,"[{'authorId': '29724362', 'name': 'Longlong Ji...",61,"[{'arxivId': None, 'authors': [{'authorId': '4...",62841734,10.1109/tpami.2020.2992393,11,4c94ee7df6bc2bfcac76703be4f059a79010f7e5,"[{'arxivId': '1803.02276', 'authors': [{'autho...",Self-supervised Visual Feature Learning with D...,"[{'topic': 'Feature learning', 'topicId': '205...",https://www.semanticscholar.org/paper/4c94ee7d...,2020.0
14571,Large-scale labeled data are generally require...,1902.06162,"[{'authorId': '29724362', 'name': 'Longlong Ji...",61,"[{'arxivId': None, 'authors':

In [1]:
import os

In [2]:
os.listdir()

['SemanticScholar_DBLP_Linkage.ipynb',
 'Matches_test.csv',
 '.DS_Store',
 'merge_vision.ipynb',
 'papers.ttl',
 'merge_vision_2.ipynb',
 'Matches.csv',
 'Parsing_Citation.ipynb',
 '.ipynb_checkpoints',
 'Citation Scraper.ipynb',
 'semantic_short.csv',
 'output.txt']

In [7]:
!pip install rdfpandas

In [3]:
from rdflib import Graph

g = Graph()
g.parse("papers.ttl", format="turtle")

<Graph identifier=N8112c80df5854c32ba2b5ecb7a2499a9 (<class 'rdflib.graph.Graph'>)>

In [11]:
import rdfpandas.graph
import pandas as pd
import rdflib

g = rdflib.Graph()
g.parse('papers.ttl', format = 'ttl')
# df.to_csv('test.csv', index = True, index_label = "@id")

NameError: name 'to_dataframe' is not defined

In [12]:
from rdfpandas.graph import to_dataframe

In [13]:
df = to_dataframe(g)


KeyboardInterrupt: 

NameError: name 'df' is not defined

In [ ]:

# NEW_ID = 0 
c_c = 0
st = datetime.now()
for index,row in df.iterrows():
#     print(index)
#     print(row)
    if index>10:
        break
    

    # row=df.iloc[2]

    NEW_ID = index

    ### URI ###
    node_uri = URIRef(str(NEW_ID))
    g.add((node_uri, RDF.type, SCHEMA.ScholarlyArticle))
    
    if  str(row['b_doi'])!="nan":
        b_doi = row['b_doi']
        g.add((node_uri, MYNS.doi, Literal(b_doi, datatype=SCHEMA.Text)))

    
    if  str(row['a_title'])!="nan":
        title = row['a_title']
    else:
        title = row['b_title']
    if title:
        g.add((node_uri, SCHEMA.headline, Literal(title, datatype=SCHEMA.Text)))


    if str(row['a_authors'])!="nan":
        authors = row['a_authors']
    else:
        authors = row['b_authors']
    if authors:
#         [g.add((node_uri, SCHEMA.author, Literal(author, datatype=SCHEMA.Person))) for author in authors]
        for author in authors:
            g.add((node_uri, SCHEMA.author, URIRef(str(author_dict[author])))) 
            
    published = None
    if str(row['b_year'])!="nan":
        published = int(row['b_year'])
    else:
        pub =row["a_published"]
        if str(pub)!="nan":
            published = datetime.strptime(pub ,"%Y-%m-%dT%H:%M:%SZ").year
        
    if published:
        g.add((node_uri, SCHEMA.datePublished, Literal(published, datatype=SCHEMA.DateTime)))

    try:
        if str(row['a_updated'])!="nan":
            updated = datetime.strptime(row["a_updated"],"%Y-%m-%dT%H:%M:%SZ").year
            g.add((node_uri, SCHEMA.dateModified, Literal(updated, datatype=SCHEMA.DateTime)))
    except:
        pass


    try:
        if str(row['b_abstract'])!="nan":
            abstract = row['b_abstract']
        else:
            abstract = row['a_abstract']
        g.add((node_uri, SCHEMA.abstract, Literal(abstract, datatype=SCHEMA.Text)))
    except:
        pass


    try:
        if str(row['a_categories'])!="nan":
            categories = ast.literal_eval(row['a_categories'])
            for category in categories:
                g.add((node_uri, SCHEMA.genre, URIRef(str(category_dict[category]))))
    except:
        pass

    if str(row['a_journal'])!="nan":
        s = row['a_journal']
        journal = ''.join([i for i in s if i.isalpha() or i==" "]).strip()
        
        g.add((node_uri, SCHEMA.publisher, URIRef(str(journal_dict[journal])))) #datatype=SCHEMA.Organisation


    if str(row['a_arxiv_url'])!="nan":
        arxiv_url = row['a_arxiv_url']

        g.add((node_uri, SCHEMA.url, Literal(arxiv_url, datatype=SCHEMA.URL)))


    if str(row['a_gscholar_url'])!="nan":
        g_scholar_url = row['a_gscholar_url']

        g.add((node_uri, SCHEMA.url, Literal(g_scholar_url, datatype=SCHEMA.URL)))

    if str(row['b_url'])!="nan":
        g_scholar_url = row['b_url']

        g.add((node_uri, SCHEMA.url, Literal(g_scholar_url, datatype=SCHEMA.URL)))

    if str(row['b_citationVelocity'])!="nan":
        num = row['b_citationVelocity']
        g.add((node_uri, MYNS.citationVelocity, Literal(num))) #datatype=SCHEMA.Organisation

    if str(row['b_influentialCitationCount'])!="nan":
        num = row['b_influentialCitationCount']
        g.add((node_uri, MYNS.b_influentialCitationCount, Literal(num))) #datatype=SCHEMA.Organisation

    try:
        citations_1 = ast.literal_eval(row['b_citations'])
    except:
        citations_1 = []
    try:
        citations_2 = row['b_dblp_info']['references_opencitations']
    except:
        citations_2 = []
    citations_1.extend(citations_2)

    for c in citations_1:

        if 'citing' in c:
            c_doi = c['citing'].split(" => ")[-1]
            citing_row = df[df['b_doi']==c_doi]
            if len(citing_row)>0:
                citing_id = citing_row.index[0]
                cite_uri = URIRef(str(citing_id))
                g.add((node_uri, SCHEMA.citation, cite_uri))
            else:
                cite_uri = c_doi
#                 len_df = len(df)
#                 cite_uri = URIRef(str(len_df))
#                 df.at[len_df,'b_doi'] = c_doi
#                 try:
#                     df.at[len_df,'b_year'] = float(c['creation'].split(" => ")[-1])
#                 except:
#                     pass
                
                g.add((node_uri, SCHEMA.citation, Literal(cite_uri)))
        else:
            try:
                c_doi = c['doi']
            except:
                c_doi = None
            if c_doi:
                citing_row = df[df['b_doi']==c_doi]
#                 if c_doi:
#                     citing_row = df[df['b_doi']==c_doi]
#                 else:
#                     citing_row = df[df['b_title']==c_title]

                if len(citing_row)>0:
                    citing_id = citing_row.index[0]
                    cite_uri = URIRef(str(citing_id))
                    g.add((node_uri, SCHEMA.citation, cite_uri))
                else:
                    cite_uri = c_doi
#                     len_df = len(df)
#                     cite_uri = URIRef(str(len_df))
#                     df.at[len_df,'b_doi'] = c_doi
#                     if c['arxivId']:
#                         df.at[len_df,'b_arxivId'] = c['arxivId']
#                     if c['authors']:
#                         df.at[len_df,'b_authors'] = [i['name'] for i in c['authors']]
#                     if c['title']:
#                         df.at[len_df,'b_title'] = c['title']
#                     if c['url']:
#                         df.at[len_df,'b_url'] = c['url']
#                     if c['year']:
#                         df.at[len_df,'b_year'] = c['year']            
                    g.add((node_uri, SCHEMA.citation, Literal(cite_uri)))
                

    #     else:
    #         g.add((node_uri, SCHEMA.citation,Literal(c_doi, datatype=SCHEMA.Text)))

    try:
        references_1 = ast.literal_eval(row['b_references'])
    except:
        referneces_1 = []
    try:
        references_2 = row['b_dblp_info']['references_crossref']
    except:
        references_2 = []
    references_1.extend(references_2)   

    for c in references_1:
        if 'key' in c:
            citing_row = []
            c_doi = None

            if "DOI" in c:
                c_doi = c["DOI"]
                citing_row = df[df['b_doi']==c_doi]

            elif "article-title" in c:
                c_title = c['article-title']
                citing_row = df[(df['b_title']==c_title) | (df['a_title']==c_title)]


            if len(citing_row)>0:
                citing_id = citing_row.index[0]
                cite_uri = URIRef(str(citing_id))
                g.add((node_uri, MYNS.references, cite_uri))
            else:
                cite_uri = c_doi 
                
#                 len_df = len(df)
#                 cite_uri = URIRef(str(len_df))

#                 try:
#                     df.at[len_df,'b_doi'] = c_doi
#                 except:
#                     pass
#                 try:
#                     df.at[len_df,'b_year'] = float(c['year'])
#                 except:
#                     pass
#                 try:
#                     df.at[len_df,'b_author'] = c['author']
#                 except:
#                     pass
#                 try:
#                     df.at[len_df,'b_title'] = c['article-title']
#                 except:
#                     pass

                g.add((node_uri, MYNS.references, Literal(cite_uri)))
        else:
            try:
                c_doi = c['doi']
            except:
                c_doi = None
                pass
            if c_doi:
                citing_row = df[df['b_doi']==c_doi]
#                 if c_doi:
#                     citing_row = df[df['b_doi']==c_doi]
#                 else:
#                     citing_row = df[df['b_title']==c_title]

                if len(citing_row)>0:
                    citing_id = citing_row.index[0]
                    cite_uri = URIRef(str(citing_id))
                    g.add((node_uri, MYNS.references, cite_uri))
                else:
                    cite_uri = c_doi
#                     len_df = len(df)
#                     cite_uri = URIRef(str(len_df))
#                     df.at[len_df,'b_doi'] = c_doi
#                     if c['arxivId']:
#                         df.at[len_df,'b_arxivId'] = c['arxivId']
#                     if c['authors']:
#                         df.at[len_df,'b_authors'] = [i['name'] for i in c['authors']]
#                     if c['title']:
#                         df.at[len_df,'b_title'] = c['title']
#                     if c['url']:
#                         df.at[len_df,'b_url'] = c['url']
#                     if c['year']:
#                         df.at[len_df,'b_year'] = c['year']  
                    g.add((node_uri, MYNS.references, Literal(cite_uri)))

    #     else:
    #         g.add((node_uri, SCHEMA.citation,Literal(c_doi, datatype=SCHEMA.Text)))
    c_c = c_c+1
    if c_c%100==0:
        et = datetime.now()
        print((et-st).total_seconds())
        print(len(df))
        print(c_c)
        

# Author Sparql

In [1]:
import sparql


In [ ]:
sparql_query_or = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX dbo:<http://dbpedia.org/ontology/>

SELECT ?lbl ?sub ?a ?b ?c ?d WHERE {
  ?sub a foaf:Person .
  ?sub rdfs:label ?lbl .
  ?lbl bif:contains "__TERM__" .
  filter(langMatches(lang(?lbl), "en"))
OPTIONAL {?sub dbo:birthName ?a}
OPTIONAL {?sub dbo:birthPlace ?b}
OPTIONAL {?sub dbo:birthDate ?c}
?sub dbo:academicDiscipline ?d

} 
limit 100
"""

In [ ]:
with open('authors.txt','r') as f:
  t = f.read()
  ls = t.split('\n')

In [ ]:

import json,pickle

In [ ]:
c=0
ans_dict = {}

In [ ]:

for r in ls[c:]:
  c+=1
    
  if c%500==0:
    with open('ans_dict.pickle', 'wb') as handle:
      print(c)
      pickle.dump(ans_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)


  line = r.split(',')

  name_res = line[0].split(" ")
  name_res = "Geoffrey Hinton".split(" ")
  search_name = ""
  search_bool = True

  if len(name_res)>1:
      for i in name_res:
          search_name = search_name +i+' AND '
          if len(i)<2 or i[1]==".":
              search_bool = False

      if search_bool:
          sparql_query = sparql_query_or[:]
          sparql_query = sparql_query.replace('__TERM__',search_name[:-5])
          result = sparql.query('http://dbpedia.org/sparql', sparql_query)

          ans_list = []
          for row in result:
              values = sparql.unpack_row(row)
              ans_list.append(values)
          if len(ans_list)>0:
              ans_dict[line[1]] = ans_list
  


In [ ]:
authors = pickle.load(open('ans_dict.pickle','rb'))

In [ ]:
import rdflib
from rdflib import URIRef, Literal, Namespace
from rdflib.namespace import RDF, RDFS, XSD

MYNS = Namespace('http://inf558.org/myfakenamespace#')
SCHEMA = Namespace("https://schema.org/")

# Initliaze the graph
g = rdflib.Graph()

# Bind namespace and prefixes
g.bind('my_ns', MYNS)
g.bind('schema', SCHEMA)
g.bind('rdf', RDF)
g.bind('rdfs', RDFS)
g.bind('xsd', XSD)

for d in tqdm(authors):
    node_uri = URIRef(d.split("/")[-1])
    g.add((node_uri, RDF.type, SCHEMA.Person))
    g.add((node_uri, SCHEMA.name, Literal(authors[d][0][2], datatype=SCHEMA.Text)))
    try:
        g.add((node_uri, SCHEMA.birthDate, Literal(authors[d][0][1], datatype=SCHEMA.Date)))
    except:
        pass
    try:
        g.add((node_uri, SCHEMA.url, Literal(authors[d][0][4], datatype=SCHEMA.URL)))
    except:
        pass

    

In [ ]:
g.serialize('authors.ttl', format="turtle")